## 1. Вход в HuggingFace Hub

In [ ]:
# Получите ваш 'HF' токен 'WRITE' здесь: 'https://huggingface.co/settings/tokens'
write_token = "hf_xxx"

# Если репо не существует, оно автоматически будет создано.
repository_name = ""    # Пустое поле даст имя для репо - 'THE_NAME_WAS_NOT_GIVEN'

# Сделать ваше репо приватным?
make_private = False    # 'True' или 'False'





# =======================================================================================
import os
import re
from huggingface_hub import login
from huggingface_hub import HfApi
from huggingface_hub.utils import validate_repo_id, HfHubHTTPError

!git config --global credential.helper store


def sanitize_model_name(name):
    # Remove invalid characters, limit the length, and fix other problems if there are any
    name = re.sub(r"[^\w.-]", "", name)
    name = name[:96]
    name = name.strip("-.")
    name = name.replace("..", ".")
    return name


def authenticate(write_token):
    login(write_token, add_to_git_credential=True)
    api = HfApi()
    return api.whoami(write_token), api


def create_repo(api, user, repo_name, repo_type, make_private=False):
    global model_repo

    repo_id = user["name"] + "/" + repo_name.strip()

    try:
        validate_repo_id(repo_id)
        api.create_repo(repo_id=repo_id, repo_type=repo_type, private=make_private)
        print(f"\nРепо '{repo_id}' не существует, создние репо.")
    except HfHubHTTPError as e:
        print(f"\nРепо '{repo_id}' существует, пропуск создания репо.")

    model_repo = repo_id
    print(f"Репо '{repo_id}' - ссылка: https://huggingface.co/{repo_id}\n")

    
user, api = authenticate(write_token)

if repository_name:
    sanitized_model_name = sanitize_model_name(repository_name)
    create_repo(api, user, repo_name=sanitized_model_name, repo_type="model", make_private=make_private)
else:
    create_repo(api, user, repo_name="THE_NAME_WAS_NOT_GIVEN", repo_type="model", make_private=make_private)

## 2. Загрузка на HuggingFace

In [ ]:
# Укажите путь к папке/файлу для загрузки. | Поддерживается также несколько путей, просто укажите их через запятую/пробел.
paths_input = ""

# Это путь директорий которые будут созданы в вашем репо. (Не обязательно)
path_in_repo = ""      # Указывайте в конце пути '/' - это важно!

# Другая информация.
commit_message = ""    # Не обязательное для заполнения.





# =======================================================================================
import re
from huggingface_hub import HfApi
from pathlib import Path

%store -r

api = HfApi()


if not commit_message:
    commit_message = "no commit message..."


def upload_to_hf_single(path, is_config):
    path_obj = Path(path)
    is_folder = path_obj.is_dir()
    files = path_in_repo + path_obj.parts[-1]

    print(f"Загрузка '{files}'")
    print("Пожалуйста, подождите... Если ничего не происходит это не значит что все встало, просто больше файлы, обрабатываются дольше...", end=" ")

    if is_folder:
        api.upload_folder(
            folder_path=path,
            path_in_repo=files,
            repo_id=model_repo,
            commit_message=commit_message,
            ignore_patterns=".ipynb_checkpoints",
        )
        print(f"\rУспешная загрузка! Расположение по ссылке - https://huggingface.co/{model_repo}/tree/main/{files}\n---\n")
    else:
        api.upload_file(
            path_or_fileobj=path,
            path_in_repo=files,
            repo_id=model_repo,
            commit_message=commit_message,
        )
        print(f"\rУспешная загрузка! Расположение по ссылке - https://huggingface.co/{model_repo}/blob/main/{files}\n---\n")


def upload_to_hf_multi(paths, is_config):
    paths = re.split(r"[,\s]+", paths)

    for path in paths:
        if not path.startswith("/home/studio-lab-user/"):
            path = "/home/studio-lab-user/" + path
        upload_to_hf_single(path, is_config)


def upload(paths_input):
    print(f"Загрузка происходит в репо '{model_repo}' | https://huggingface.co/{model_repo}\n\n")

    if "," in paths_input or " " in paths_input:
        upload_to_hf_multi(paths_input, False)
    else:
        if not paths_input.startswith("/home/studio-lab-user/"):
            paths_input = "/home/studio-lab-user/" + paths_input
        upload_to_hf_single(paths_input, False)


upload(paths_input)

## ---